# How to add SharedMemory to an Agent and its Tools

This notebook goes over adding memory to **both** of an Agent and its tools. Before going through this notebook, please walk through the following notebooks, as this will build on top of both of them:

- [Adding memory to an LLM Chain](../../memory/examples/adding_memory.ipynb)
- [Custom Agents](custom_agent.ipynb)

We are going to create a custom Agent. The agent has access to a conversation memory, search tool, and a summarization tool. And, the summarization tool also needs access to the conversation memory.

In [1]:
import sys
sys.executable

'c:\\Users\\jonma\\github_repos\\jonmatthis\\golem_garden\\venv\\Scripts\\python.exe'

In [9]:
# %pip install langchain --quiet
# %pip install openai --quiet
# %pip install python-dotenv --quiet
# %pip install transformers --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os 
from dotenv import load_dotenv
load_dotenv()


'af2aa826e182072594dd5549fbf52be2341fd910'

In [12]:

from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.utilities import GoogleSerperAPIWrapper

In [10]:
template = """This is a conversation between a human and a bot:

{chat_history}

Write a summary of the conversation for {input}:
"""

prompt = PromptTemplate(
    input_variables=["input", "chat_history"], 
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")
readonlymemory = ReadOnlySharedMemory(memory=memory)
summry_chain = LLMChain(
    llm=OpenAI(), 
    prompt=prompt, 
    verbose=True, 
    memory=readonlymemory, # use the read-only memory to prevent the tool from modifying the memory
)

In [13]:
search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name = "Summary",
        func=summry_chain.run,
        description="useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary."
    )
]

In [15]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "chat_history", "agent_scratchpad"]
)


input_variables=['input', 'chat_history', 'agent_scratchpad'] output_parser=None partial_variables={} template='Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:\n\nSearch: useful for when you need to answer questions about current events\nSummary: useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Summary]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!"\n\n{chat_history}\nQuestion: {input}\n{agent_scratchpad}' template_format='f-string' validate

We can now construct the LLMChain, with the Memory object, and then create the agent.

In [16]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [17]:
agent_chain.run(input="What is ChatGPT?")



> Entering new AgentExecutor chain...
Thought: I should research ChatGPT to answer this question.
Action: Search
Action Input: "ChatGPT"
Observation: ChatGPT: Software. ChatGPT is an artificial-intelligence chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3.5 and GPT-4 families of large language models and has been fine-tuned using both supervised and reinforcement... ChatGPT Initial release date: November 30, 2022. ChatGPT Developer(s): OpenAI. ChatGPT License: Proprietary. ChatGPT Stable release: March 23, 2023; 19 days ago. ChatGPT is a sibling model to InstructGPT, which is trained to follow an instruction in a prompt and provide a detailed response. We've trained a model called ChatGPT which interacts in a conversational way. The dialogue format makes it possible for ChatGPT to answer follow-up ... ChatGPT is an artificial-intelligence (AI) chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3

"ChatGPT is an artificial-intelligence chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3.5 and GPT-4 families of large language models and has been fine-tuned using both supervised and reinforcement learning. It is a natural language processing tool driven by AI technology that allows you to have human-like conversations and much more with the AI."

To test the memory of this agent, we can ask a followup question that relies on information in the previous exchange to be answered correctly.

In [18]:
agent_chain.run(input="Who developed it?")



> Entering new AgentExecutor chain...
Thought: I need to find out who developed ChatGPT
Action: Search
Action Input: Who developed ChatGPT
Observation: ChatGPT is an artificial-intelligence (AI) chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3.5 and GPT-4 families of large language models (LLMs) and has been fine-tuned (an approach to transfer learning) using both supervised and reinforcement learning techniques.
Thought: I now know the answer
Final Answer: ChatGPT was developed by OpenAI.

> Finished chain.


'ChatGPT was developed by OpenAI.'

In [19]:
agent_chain.run(input="Thanks. Summarize the conversation, for my daughter 5 years old.")



> Entering new AgentExecutor chain...
Thought: I need to simplify the conversation for a 5 year old.
Action: Summary
Action Input: My daughter 5 years old

> Entering new LLMChain chain...
Prompt after formatting:
This is a conversation between a human and a bot:

Human: What is ChatGPT?
AI: ChatGPT is an artificial-intelligence chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3.5 and GPT-4 families of large language models and has been fine-tuned using both supervised and reinforcement learning. It is a natural language processing tool driven by AI technology that allows you to have human-like conversations and much more with the AI.
Human: Who developed it?
AI: ChatGPT was developed by OpenAI.

Write a summary of the conversation for My daughter 5 years old:


> Finished chain.

Observation: 
The conversation is about ChatGPT, an AI chatbot developed by OpenAI. It uses AI technology to have conversations like humans do. OpenAI developed 

'ChatGPT is an AI chatbot developed by OpenAI that can have conversations like humans do.'

Confirm that the memory was correctly updated.

In [ ]:
print(agent_chain.memory.buffer)

For comparison, below is a bad example that uses the same memory for both the Agent and the tool.

In [20]:
## This is a bad practice for using the memory.
## Use the ReadOnlySharedMemory class, as shown above.

template = """This is a conversation between a human and a bot:

{chat_history}

Write a summary of the conversation for {input}:
"""

prompt = PromptTemplate(
    input_variables=["input", "chat_history"], 
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")
summry_chain = LLMChain(
    llm=OpenAI(), 
    prompt=prompt, 
    verbose=True, 
    memory=memory,  # <--- this is the only change
)

search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name = "Summary",
        func=summry_chain.run,
        description="useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary."
    )
]

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "chat_history", "agent_scratchpad"]
)

llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [21]:
agent_chain.run(input="What is ChatGPT?")



> Entering new AgentExecutor chain...
Thought: I should research ChatGPT to answer this question.
Action: Search
Action Input: "ChatGPT"
Observation: ChatGPT: Software. ChatGPT is an artificial-intelligence chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3.5 and GPT-4 families of large language models and has been fine-tuned using both supervised and reinforcement... ChatGPT Initial release date: November 30, 2022. ChatGPT Developer(s): OpenAI. ChatGPT License: Proprietary. ChatGPT Stable release: March 23, 2023; 19 days ago. ChatGPT is a sibling model to InstructGPT, which is trained to follow an instruction in a prompt and provide a detailed response. We've trained a model called ChatGPT which interacts in a conversational way. The dialogue format makes it possible for ChatGPT to answer follow-up ... ChatGPT is an artificial-intelligence (AI) chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3

"ChatGPT is an artificial-intelligence chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3.5 and GPT-4 families of large language models and has been fine-tuned using both supervised and reinforcement learning. It is designed to produce human-like text and converse with people."

In [22]:
agent_chain.run(input="Who developed it?")



> Entering new AgentExecutor chain...
Thought: I need to find out who developed ChatGPT
Action: Search
Action Input: Who developed ChatGPT
Observation: ChatGPT is an artificial-intelligence (AI) chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3.5 and GPT-4 families of large language models (LLMs) and has been fine-tuned (an approach to transfer learning) using both supervised and reinforcement learning techniques.
Thought: I now know the answer
Final Answer: ChatGPT was developed by OpenAI.

> Finished chain.


'ChatGPT was developed by OpenAI.'

In [23]:
agent_chain.run(input="Thanks. Summarize the conversation, for my daughter 5 years old.")



> Entering new AgentExecutor chain...
Thought: I need to summarize the conversation in a way that a 5 year old can understand.
Action: Summary
Action Input: My daughter 5 years old

> Entering new LLMChain chain...
Prompt after formatting:
This is a conversation between a human and a bot:

Human: What is ChatGPT?
AI: ChatGPT is an artificial-intelligence chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3.5 and GPT-4 families of large language models and has been fine-tuned using both supervised and reinforcement learning. It is designed to produce human-like text and converse with people.
Human: Who developed it?
AI: ChatGPT was developed by OpenAI.

Write a summary of the conversation for My daughter 5 years old:


> Finished chain.

Observation: 
The conversation was about ChatGPT, a chatbot developed by OpenAI. It was created using GPT-3.5 and GPT-4 language models and can generate human-like text and have conversations.
Thought: I now 

'ChatGPT is a chatbot created by OpenAI that can talk and understand people.'

The final answer is not wrong, but we see the 3rd Human input is actually from the agent in the memory because the memory was modified by the summary tool.

In [24]:
print(agent_chain.memory.buffer)

Human: What is ChatGPT?
AI: ChatGPT is an artificial-intelligence chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3.5 and GPT-4 families of large language models and has been fine-tuned using both supervised and reinforcement learning. It is designed to produce human-like text and converse with people.
Human: Who developed it?
AI: ChatGPT was developed by OpenAI.
Human: My daughter 5 years old
AI: 
The conversation was about ChatGPT, a chatbot developed by OpenAI. It was created using GPT-3.5 and GPT-4 language models and can generate human-like text and have conversations.
Human: Thanks. Summarize the conversation, for my daughter 5 years old.
AI: ChatGPT is a chatbot created by OpenAI that can talk and understand people.
